In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pip install tensorflow opencv-python moviepy scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import os

In [4]:

def extract_frames(video_path, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imwrite(os.path.join(output_folder, f"frame_{frame_count:04d}.jpg"), frame)
        frame_count += 1
    cap.release()

# Example usage
extract_frames(r"C:\Users\101ri\OneDrive\Desktop\deep fake detector\Notebook\Celeb-real\id56_0004.mp4", "Celeb-real/frames")
extract_frames(r"C:\Users\101ri\OneDrive\Desktop\deep fake detector\Notebook\Celeb-synthesis\id59_id5_0004.mp4", "Celeb-synthesis/frames")


In [5]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [6]:

def load_images_from_folder(folder, label, image_size=(128, 128)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=image_size)
        img_array = img_to_array(img)
        images.append(img_array)
        labels.append(label)
    return images, labels

fake_images, fake_labels = load_images_from_folder("Celeb-real/frames", label=0)  # 0 for real
real_images, real_labels = load_images_from_folder("Celeb-synthesis/frames", label=1)  # 1 for fake

X = np.array(real_images + fake_images)
y = np.array(real_labels + fake_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
X_train

array([[[[127., 111.,  88.],
         [121., 105.,  79.],
         [ 88.,  85.,  68.],
         ...,
         [ 74.,  65.,  48.],
         [ 74.,  65.,  48.],
         [ 73.,  64.,  47.]],

        [[127., 116.,  94.],
         [123., 113.,  88.],
         [ 73.,  66.,  50.],
         ...,
         [ 74.,  65.,  48.],
         [ 74.,  65.,  48.],
         [ 73.,  64.,  47.]],

        [[124., 112.,  96.],
         [126., 115.,  95.],
         [ 76.,  69.,  53.],
         ...,
         [ 74.,  65.,  48.],
         [ 74.,  65.,  48.],
         [ 73.,  64.,  47.]],

        ...,

        [[ 35.,  31.,  22.],
         [ 34.,  30.,  21.],
         [ 27.,  27.,  15.],
         ...,
         [ 71.,  74.,  53.],
         [ 69.,  72.,  51.],
         [123., 126., 105.]],

        [[ 36.,  32.,  23.],
         [ 32.,  28.,  19.],
         [ 26.,  26.,  14.],
         ...,
         [ 75.,  78.,  57.],
         [134., 137., 116.],
         [138., 141., 120.]],

        [[ 36.,  32.,  23.],
       

In [8]:
### frames extraction

In [9]:
from moviepy.editor import VideoFileClip

In [10]:
def extract_frames_from_video(video_path, frame_count=30):
    clip = VideoFileClip(video_path)
    frames = []
    duration = clip.duration
    for i in range(frame_count):
        frame = clip.get_frame(i * duration / frame_count)
        frame = cv2.resize(frame, (128, 128))
        frames.append(frame)
    return np.array(frames)

In [11]:
print(extract_frames_from_video(r"C:\Users\101ri\OneDrive\Desktop\deep fake detector\Notebook\Celeb-real\id0_0000.mp4", 30))

[[[[ 97   0   0]
   [ 95   0   0]
   [ 90   1   0]
   ...
   [ 29  13   7]
   [ 29  16   9]
   [ 33  20  13]]

  [[100   2   1]
   [101   3   2]
   [ 99   1   2]
   ...
   [ 29  13   7]
   [ 29  16   9]
   [ 33  20  13]]

  [[ 86   3   0]
   [ 87   4   0]
   [ 88   2   0]
   ...
   [ 29  13   7]
   [ 29  16   9]
   [ 33  20  13]]

  ...

  [[ 65   0   0]
   [ 65   0   0]
   [ 65   0   0]
   ...
   [ 29  11   6]
   [ 29  11   6]
   [ 29  11   6]]

  [[ 65   0   0]
   [ 65   0   0]
   [ 65   0   0]
   ...
   [ 29  11   6]
   [ 29  11   6]
   [ 29  11   6]]

  [[ 62   0   0]
   [ 62   0   0]
   [ 62   0   0]
   ...
   [ 29  11   6]
   [ 31  10   6]
   [ 31  10   6]]]


 [[[ 96   0   0]
   [ 97   1   1]
   [ 91   0   1]
   ...
   [ 29  13   7]
   [ 31  15   9]
   [ 30  19  12]]

  [[103   1   3]
   [104   2   4]
   [ 99   0   1]
   ...
   [ 29  13   7]
   [ 31  15   9]
   [ 30  19  12]]

  [[ 91   4   1]
   [ 90   3   1]
   [ 87   3   0]
   ...
   [ 29  13   7]
   [ 31  15   9]
   [ 31  20

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [13]:
from tensorflow.keras.callbacks import EarlyStopping


### CNN model

In [14]:
def build_cnn_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, trainable=False),  # Frozen layer
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.6),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [15]:
X_train.size

32587776

In [16]:
X_test.size

8159232

In [17]:
y_train.size

663

In [18]:
y_test.size

166

In [19]:
input_shape = (128, 128, 3)  # Adjust based on your frame size
cnn_model = build_cnn_model(input_shape)

In [20]:
cnn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 14, 14, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       3,211,392 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,303,873 (12.60 MB)

 Non-trainable params: 896 (3.50 KB)

In [21]:
# Train the model
cnn_model.fit(X_train, y_train, epochs=42, batch_size=36, validation_split=0.3)

Epoch 1/42
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 174ms/step - accuracy: 0.7950 - loss: 7.1640 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/42
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 152ms/step - accuracy: 0.9995 - loss: 0.0088 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/42
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 153ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/42
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - accuracy: 0.9981 - loss: 0.0296 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/42
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - accuracy: 0.9990 - loss: 0.0329 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 6/42
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 150ms/step - accuracy: 1.0000 - loss: 8.5544e-14 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 7/42
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 149ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 8/42
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 151ms/step - accuracy: 

In [22]:
from sklearn.metrics import accuracy_score

In [23]:
# model evalution
loss, accuracy = cnn_model.evaluate(X_test, y_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 1.0000 - loss: 0.0000e+00


In [24]:
print(f'Cnn Model Test Accuracy: {accuracy}')

Cnn Model Test Accuracy: 1.0


In [25]:
cnn_model.save("cnn_model.h5")

### LSTM

In [26]:
from tensorflow.keras.layers import LSTM, TimeDistributed, Bidirectional


In [27]:
# Assuming X_train has shape (num_samples, 128, 128, 3)
# and you want to create sequences of 10 frames
num_frames_per_sequence = 10

# Calculate the number of sequences you can create
num_sequences = len(X_train) // num_frames_per_sequence

# Reshape data into sequences
X_train_lstm = X_train[:num_sequences * num_frames_per_sequence]
X_train_lstm = X_train_lstm.reshape((num_sequences, num_frames_per_sequence, 128, 128, 3))

# Repeat similarly for X_test


In [28]:
from tensorflow.keras import layers

def build_lstm_model(input_shape):
    model = Sequential([
        TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape)),
        TimeDistributed(layers.MaxPooling2D((2, 2))),
        TimeDistributed(layers.Flatten()),
        Bidirectional(layers.LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        Bidirectional(layers.LSTM(50, dropout=0.2, recurrent_dropout=0.2)),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [29]:
input_shape = (10, 128, 128, 3)  # Example with 10 frames per sequence
lstm_model = build_lstm_model(input_shape)

In [30]:
# Prepare data for LSTM (e.g., sequences of frames)
# X_train_lstm, X_test_lstm need to be prepared with sequences of frames

# Train the model
lstm_model.fit(X_train_lstm, y_train, epochs=50, batch_size=8, validation_split=0.2)


Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 23s 2s/step - accuracy: 0.5174 - loss: 0.7186 - val_accuracy: 0.2857 - val_loss: 0.7763
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5774 - loss: 0.6966 - val_accuracy: 0.3571 - val_loss: 0.7029
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.6059 - loss: 0.6680 - val_accuracy: 0.2857 - val_loss: 0.8731
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.4767 - loss: 0.7640 - val_accuracy: 0.2857 - val_loss: 0.8135
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5433 - loss: 0.6998 - val_accuracy: 0.2857 - val_loss: 0.8531
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.5660 - loss: 0.6915 - val_accuracy: 0.2857 - val_loss: 0.8837
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.6026 - loss: 0.6864 - val_accuracy: 0.2857 - val_loss: 0.7677
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.4710 - loss: 0.6992 - val_accuracy: 0.5000 - val_loss: 0.6991
Epoch 9/

In [31]:
lstm_model.save("lstm_model.h5")

Almost 60-65

### Capsule Network

In [32]:
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dense, Lambda, TimeDistributed, Flatten, LSTM
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K

In [33]:
## forcing tensorflow to use gpu
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [34]:
## clear previous session
K.clear_session()

In [35]:
## custom squash function for capsule layers
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return s_squared_norm / (1. + s_squared_norm) * x / scale

In [36]:
## capsule network model
def build_capsule_network(input_shape):
    inputs = Input(shape=input_shape)
    conv1 = TimeDistributed(Conv2D(128, (9, 9), activation='relu'))(inputs)  # Reduced filters
    primary_caps = TimeDistributed(Conv2D(8 * 16, (9, 9), strides=2, padding='valid'))(conv1)  # Reduced filters
    primary_caps = TimeDistributed(Reshape((-1, 8)))(primary_caps)
    primary_caps = TimeDistributed(Lambda(squash))(primary_caps)
    
    primary_caps_flat = TimeDistributed(Flatten())(primary_caps)
    lstm_output = LSTM(50)(primary_caps_flat)
    
    caps_output = Dense(1, activation='sigmoid')(lstm_output)
    
    model = Model(inputs=inputs, outputs=caps_output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [37]:
X_train = np.random.rand(300, 128, 128, 3)  # Reduced number of frames
y_train = np.random.randint(0, 2, size=(30,))  # Corresponding labels

In [38]:
## reshape the data into sequences of frames
num_frames_per_sequence = 10
num_sequences = len(X_train) // num_frames_per_sequence

X_train_lstm = X_train[:num_sequences * num_frames_per_sequence].reshape(
    (num_sequences, num_frames_per_sequence, 128, 128, 3)
)

y_train_lstm = y_train[:num_sequences]

In [39]:
input_shape = (num_frames_per_sequence, 128, 128, 3)
capsule_model = build_capsule_network(input_shape)

In [40]:
capsule_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 10, 128, 128, 3)     │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed (TimeDistributed)   │ (None, 10, 120, 120, 128)   │          31,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_1 (TimeDistributed) │ (None, 10, 56, 56, 128)     │       1,327,232 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 10, 50176, 8)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_3 (TimeDistributed) │ (None, 10, 50176, 8)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 10, 401408)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 50)                  │      80,291,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 81,650,315 (311.47 MB)

 Trainable params: 81,650,315 (311.47 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
capsule_model.fit(X_train_lstm, y_train_lstm, epochs=50, batch_size=8, validation_split=0.2)

Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 56s 16s/step - accuracy: 0.3594 - loss: 1.3550 - val_accuracy: 0.3333 - val_loss: 0.7109
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 13s/step - accuracy: 0.6250 - loss: 0.6786 - val_accuracy: 0.3333 - val_loss: 0.8256
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 14s/step - accuracy: 0.6250 - loss: 0.6629 - val_accuracy: 0.3333 - val_loss: 0.7700
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 40s 13s/step - accuracy: 0.6562 - loss: 0.7087 - val_accuracy: 0.3333 - val_loss: 0.8330
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 13s/step - accuracy: 0.6094 - loss: 0.6752 - val_accuracy: 0.3333 - val_loss: 0.8535
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 39s 13s/step - accuracy: 0.6719 - loss: 0.6334 - val_accuracy: 0.3333 - val_loss: 0.8532
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 38s 13s/step - accuracy: 0.6875 - loss: 0.6232 - val_accuracy: 0.3333 - val_loss: 0.8526
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 41s 14s/step - accuracy: 0.6250 - loss: 0.6637 - val_accuracy: 0.3333 - val_loss: 0.8506


In [42]:
# model evalution
loss, accuracy = capsule_model.evaluate(X_train_lstm, y_train_lstm)
print(f'Cnn Model Test Accuracy: {accuracy}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.5667 - loss: 0.6920
Cnn Model Test Accuracy: 0.5666666626930237


In [43]:
import keras

In [44]:
keras.saving.save_model(capsule_model, 'capsule_model.keras')

In [45]:
pip install keras

In [46]:
stop123

NameError: name 'stop123' is not defined

### Gans

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, LeakyReLU, Dropout

In [ ]:
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Sequential

### Generator Model

In [ ]:
latent_dim = 20  # Example latent dimension

def build_generator(latent_dim):
    model_gen = Sequential()
    
    # Fully connected layer
    model_gen.add(Dense(16 * 16 * 256, input_dim=latent_dim))
    model_gen.add(Reshape((16, 16, 256)))
    
    # Transposed Convolutional Layers to upscale to (128, 128, 3)
    model_gen.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model_gen.add(BatchNormalization())
    model_gen.add(LeakyReLU(alpha=0.2))
    
    model_gen.add(Conv2DTranspose(64, (4, 4), strides=(2, 2), padding='same'))
    model_gen.add(BatchNormalization())
    model_gen.add(LeakyReLU(alpha=0.2))
    
    model_gen.add(Conv2DTranspose(3, (4, 4), strides=(2, 2), padding='same', activation='tanh'))
    
    return model_gen

generator = build_generator(latent_dim)

In [ ]:
def build_discriminator(input_shape=(128, 128, 3)):
    model_dis = Sequential()
    
    # Convolutional Layers
    model_dis.add(Conv2D(64, (3, 3), padding='same', input_shape=input_shape))
    model_dis.add(LeakyReLU(alpha=0.2))
    
    model_dis.add(Conv2D(128, (3, 3), padding='same'))
    model_dis.add(LeakyReLU(alpha=0.2))
    
    model_dis.add(Flatten())
    model_dis.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation
    
    return model_dis

discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, BatchNormalization, Activation

In [ ]:
## build and compile ghans

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

def build_gan(generator, discriminator):
    discriminator.trainable = False  # Ensure the discriminator is not trainable within the GAN
    
    gan_input = Input(shape=(latent_dim,))
    img = generator(gan_input)
    gan_output = discriminator(img)
    
    gan = Model(gan_input, gan_output)
    gan.compile(optimizer='adam', loss='binary_crossentropy')
    
    return gan

gan = build_gan(generator, discriminator)


In [ ]:
gan.summary()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_real_samples(X_train, n_samples):
    idx = np.random.randint(0, X_train.shape[0], n_samples)
    X = X_train[idx]
    y = np.ones((n_samples, 1))  # Label as real (1)
    return X, y

def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = np.random.randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))  # Label as fake (0)
    return X, y

def save_plot(examples, epoch, n=5):
    examples = (examples + 1) / 2.0  # Scale from [-1, 1] to [0, 1]
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        plt.imshow(examples[i])
    plt.savefig(f"generated_plot_e{epoch+1}.png")
    plt.close()

def train_gan(gan, generator, discriminator, X_train, latent_dim, n_epochs=100, n_batch=64, eval_interval=20):
    batch_per_epoch = X_train.shape[0] // n_batch
    half_batch = n_batch // 2

    for epoch in range(n_epochs):
        for _ in range(batch_per_epoch):
            # Train Discriminator on real and fake images
            X_real, y_real = generate_real_samples(X_train, half_batch)
            X_fake, y_fake = generate_fake_samples(generator, latent_dim, half_batch)
            d_loss_real = discriminator.train_on_batch(X_real, y_real)
            d_loss_fake = discriminator.train_on_batch(X_fake, y_fake)

            # Train Generator via the GAN
            x_gan = np.random.randn(latent_dim * n_batch)
            x_gan = x_gan.reshape(n_batch, latent_dim)
            y_gan = np.ones((n_batch, 1))  # Attempt to fool the discriminator
            g_loss = gan.train_on_batch(x_gan, y_gan)

        # Evaluate and save model performance at set intervals
        if (epoch + 1) % eval_interval == 0:
            print(f"Epoch {epoch + 1}/{n_epochs} - D loss real: {d_loss_real}, D loss fake: {d_loss_fake}, G loss: {g_loss}")
            X_fake, _ = generate_fake_samples(generator, latent_dim, 25)
            save_plot(X_fake, epoch)

# Assume X_train is your training dataset loaded here
# X_train = ...

train_gan(gan, generator, discriminator, X_train, latent_dim)

In [ ]:
model_gan.save('model_gan.h5')
model_dis.save('model_dis.h5')

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input

In [ ]:
def create_frame_sequences(frames, sequence_length):
    sequences = []
    for i in range(len(frames) - sequence_length + 1):
        sequences.append(frames[i:i + sequence_length])
    return np.array(sequences)

In [ ]:
# Define the sequence length (e.g., 10 frames per sequence)
sequence_length = 10

In [ ]:
# Prepare sequential data for LSTM
X_train_lstm = create_frame_sequences(X_train, sequence_length)
X_test_lstm = create_frame_sequences(X_test, sequence_length)

In [ ]:
X_test_lstm

In [ ]:
# Adjust y_train and y_test for the LSTM sequences
y_train_lstm = y_train[sequence_length - 1:]  # Match the sequence length
y_test_lstm = y_test[sequence_length - 1:]

### Final evalution of all models

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

## CNN Evaluation
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test)
print(f"CNN Loss: {cnn_loss}, Accuracy: {cnn_accuracy}")

## LSTM Evaluation
lstm_loss, lstm_accuracy = lstm_model.evaluate(X_test_lstm, y_test_lstm)
print(f"LSTM Loss: {lstm_loss}, Accuracy: {lstm_accuracy}")

## capsule Network Evaluation
capsule_loss, capsule_accuracy = capsule_model.evaluate(X_test, y_test)
print(f"Capsule Network Loss: {capsule_loss}, Accuracy: {capsule_accuracy}")

## classification Report and Confusion Matrix
y_pred_cnn = (cnn_model.predict(X_test) > 0.5).astype("int32")
y_pred_lstm = (lstm_model.predict(X_test_lstm) > 0.5).astype("int32")
y_pred_capsule = (capsule_model.predict(X_test) > 0.5).astype("int32")

print("CNN Classification Report:\n", classification_report(y_test, y_pred_cnn))
print("CNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred_cnn))

print("LSTM Classification Report:\n", classification_report(y_test_lstm, y_pred_lstm))
print("LSTM Confusion Matrix:\n", confusion_matrix(y_test_lstm, y_pred_lstm))

print("Capsule Network Classification Report:\n", classification_report(y_test, y_pred_capsule))
print("Capsule Network Confusion Matrix:\n", confusion_matrix(y_test, y_pred_capsule))


### Audio- Video

In [ ]:
import cv2
import os

In [ ]:
def extract_frames(video_path, output_folder, frame_rate=1):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    count = 0
    success, frame = cap.read()
    while success:
        if count % frame_rate == 0:
            cv2.imwrite(os.path.join(output_folder, f"frame_{count:04d}.jpg"), frame)
        success, frame = cap.read()
        count += 1
    cap.release()

# Example usage
extract_frames(r"C:\Users\101ri\OneDrive\Desktop\deep fake detector\Notebook\Celeb-real\id0_0000.mp4", "video_frames")

In [ ]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

def extract_audio(video_path, output_audio_path):
    os.system(f"ffmpeg -i {video_path} -q:a 0 -map a {output_audio_path}")

def audio_to_melspectrogram(audio_path):
    y, sr = librosa.load(audio_path)
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    log_S = librosa.power_to_db(S, ref=np.max)
    return log_S, sr

def save_melspectrogram(log_S, output_path):
    plt.figure(figsize=(10, 4))
    librosa.display.specshow(log_S, sr=22050, x_axis='time', y_axis='mel')
    plt.colorbar(format='%+02.0f dB')
    plt.savefig(output_path)
    plt.close()

# Example usage
extract_audio(r"C:\Users\101ri\OneDrive\Desktop\deep fake detector\Notebook\Celeb-real\id0_0001.mp4", "audio.wav")
log_S, sr = audio_to_melspectrogram("audio.wav")
save_melspectrogram(log_S, "melspectrogram.png")


In [ ]:
ffmpeg -version
